In [27]:
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np
import cv2
import glob

batch_size = 100
num_epochs = 9
kernel_size = 7 # baslangic icin n3x3, her epoch da 2ser artacak. Bir sinirin ustune cikince basariyi hic artirmiyor tikaniyor
pool_size = 2 # 2x2 eger cok buyuk olursa resmin pikselleri anlam kaybeder
conv_depth_1 = 32 # 
conv_depth_2 = 64 # 
conv_depth_3 = 128
drop_prob_1 = 0.2 # bagnazligin olusmasini engelliyor
drop_prob_2 = 0.4 # 
hidden_size = 1024 # bagli olan gizli etken sayisi, ne cok ne az olmasi gerek
w,h = 50,50

In [13]:
X_train = []
y_train = []


X_test = []
y_test = []

i = 0
for filename in glob.glob('E:/cancer_3000/mal/*.png'): #assuming gif
    im = cv2.imread(filename)
    im = cv2.resize(im, (w, h)) 
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    
    if(i >2400):
        X_test.append(im)
        y_test.append(0)
    else:
        X_train.append(im)
        y_train.append(0)
    i += 1
i = 0
for filename in glob.glob('E:/cancer_3000/kidney/*.png'): #assuming gif
    im = cv2.imread(filename)
    im = cv2.resize(im, (w, h)) 
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    
    if(i > 2400):
        X_test.append(im)
        y_test.append(1)
    else:
        X_train.append(im)
        y_train.append(1)    
    i += 1
i = 0
for filename in glob.glob('E:/cancer_3000/skin/*.png'): #assuming gif
    im = cv2.imread(filename)
    im = cv2.resize(im, (w, h)) 
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    
    if(i > 2400):
        X_test.append(im)
        y_test.append(2)
    else:
        X_train.append(im)
        y_train.append(2)    
    i += 1
i = 0
for filename in glob.glob('E:/cancer_3000/lung/*.png'): #assuming gif
    im = cv2.imread(filename)
    im = cv2.resize(im, (w, h)) 
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    
    if(i > 2400):
        X_test.append(im)
        y_test.append(3)
    else:
        X_train.append(im)
        y_train.append(3)    
    i += 1
i = 0
for filename in glob.glob('E:/cancer_3000/H&N/*.png'): #assuming gif
    im = cv2.imread(filename)
    im = cv2.resize(im, (w, h)) 
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    
    if(i > 2400):
        X_test.append(im)
        y_test.append(4)
    else:
        X_train.append(im)
        y_train.append(4)    
    i += 1

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(np.unique(y_train).shape[0])

num_train, height, width, depth = X_train.shape #
print(num_train)
print(height)
print(width)
print(depth)
num_test = X_test.shape[0] # 
print(num_test)
num_classes = np.unique(y_train).shape[0] # sinif sayisi (kanser ya da degil)
print(num_classes)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255 # Normalizasyon
X_test /= 255 # 

 


Y_train = np_utils.to_categorical(y_train, num_classes) # 2 -> [0, 0, 1]
Y_test = np_utils.to_categorical(y_test, num_classes) # 


5
12005
50
50
3
2995
5


In [14]:


inp = Input(shape=(height, width, depth)) # Input layer

# BLOCK 1

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
kernel_size -= 2 # 5x5  correct 7x7
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
kernel_size -= 2 # 7x7    correct 5x5
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)o
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
kernel_size -= 2 # 9x9
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2) # anti bagnaz
# my change
# Conv [128] -> Conv [128] -> Pool (with dropout on the pooling layer)

#conv_5 = Convolution2D(conv_depth_3, (kernel_size, kernel_size), padding='same', activation='relu')(drop_2)
#kernel_size -= 2 # 9x9
#conv_6 = Convolution2D(conv_depth_3, (kernel_size, kernel_size), padding='same', activation='relu')(conv_5)
#pool_3 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_6)
#drop_3 = Dropout(drop_prob_1)(pool_3) # anti bagnaz

# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_4 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_4) # softmax RESIM!!!!

model = Model(inputs=inp, outputs=out) # Eger 0 dan insa edilecekse

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='Adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train,                 
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1) 




Train on 10804 samples, validate on 1201 samples
Epoch 1/9
10804/10804 [==============================] - 388s 36ms/step - loss: 0.9852 - acc: 0.5749 - val_loss: 0.9540 - val_acc: 0.2448
Epoch 2/9
10804/10804 [==============================] - 370s 34ms/step - loss: 0.6560 - acc: 0.7382 - val_loss: 4.9670 - val_acc: 0.0000e+00
Epoch 3/9
10804/10804 [==============================] - 369s 34ms/step - loss: 0.3790 - acc: 0.8508 - val_loss: 0.1715 - val_acc: 0.9950
Epoch 4/9
10804/10804 [==============================] - 371s 34ms/step - loss: 0.2865 - acc: 0.8916 - val_loss: 0.0557 - val_acc: 0.9983
Epoch 5/9
10804/10804 [==============================] - 372s 34ms/step - loss: 0.2396 - acc: 0.9111 - val_loss: 0.0014 - val_acc: 1.0000
Epoch 6/9
10804/10804 [==============================] - 371s 34ms/step - loss: 0.4678 - acc: 0.8232 - val_loss: 0.2770 - val_acc: 0.9509
Epoch 7/9
10804/10804 [==============================] - 375s 35ms/step - loss: 0.2288 - acc: 0.9117 - val_loss: 0.0455

[0.1729116788740349, 0.9429048414023372]

ValueError: Error when checking input: expected input_6 to have 4 dimensions, but got array with shape (0, 1)

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./beng/8974_idx5_x3451_y851_class0.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

plt.figure(1, figsize=(4, 4))
plt.imshow(img, cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()
img = cv2.resize(img, (w, h)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 

 
out2 = model.predict(img)

print(str(np.argmax(out2, axis=1))+" <-- beng")

error: OpenCV(4.0.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [15]:
model_json = model.to_json()
with open("kidskinbrelunghn2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("kidskinbrelunghn2.h5")
print("Saved model to disk")


Saved model to disk


In [28]:
from keras.models import model_from_json
json_file = open('kidskinbrelunghn1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("kidskinbrelunghn1.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
from keras.models import model_from_json
json_file = open('bre.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("bre.h5")
print("Loaded model from disk")

C:\Users\MY HOME\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Loaded model from disk


In [7]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1)  # Evalu

Train on 8176 samples, validate on 909 samples
Epoch 1/8
8176/8176 [==============================] - 464s 57ms/step - loss: 0.3650 - acc: 0.8460 - val_loss: 0.5561 - val_acc: 0.7558
Epoch 2/8
8176/8176 [==============================] - 355s 43ms/step - loss: 0.3285 - acc: 0.8662 - val_loss: 0.5915 - val_acc: 0.7459
Epoch 3/8
8176/8176 [==============================] - 340s 42ms/step - loss: 0.3120 - acc: 0.8711 - val_loss: 0.3480 - val_acc: 0.9307
Epoch 4/8
8176/8176 [==============================] - 336s 41ms/step - loss: 0.3094 - acc: 0.8730 - val_loss: 0.5349 - val_acc: 0.7580
Epoch 5/8
8176/8176 [==============================] - 349s 43ms/step - loss: 0.3083 - acc: 0.8722 - val_loss: 0.8201 - val_acc: 0.5897
Epoch 6/8
8176/8176 [==============================] - 344s 42ms/step - loss: 0.3034 - acc: 0.8754 - val_loss: 0.5611 - val_acc: 0.7602
Epoch 7/8
8176/8176 [==============================] - 337s 41ms/step - loss: 0.3125 - acc: 0.8721 - val_loss: 0.6028 - val_acc: 0.7217
E

[0.2563134040042642, 0.8592304846549581]

In [7]:
for layer in model.layers:
    weights = layer.get_weights()
print(weights)

[array([[-0.08496416,  0.06198893],
       [-0.02103596, -0.02947552],
       [ 0.0604041 ,  0.00327168],
       ...,
       [-0.00782773, -0.04730971],
       [ 0.02001816,  0.03351046],
       [ 0.06193078,  0.04109517]], dtype=float32), array([-0.04418868,  0.04418867], dtype=float32)]


In [8]:
print(X_train.shape)

(24002, 50, 50, 3)


In [36]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2
from glob import glob                                                           
import os
import simpleaudio as sa
src = glob('F:/dataset/*.png')           # correct code
for j in src:
    print(j)
    img = cv2.imread(j)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (w, h)) 
    img = img.astype('float32')
    img /= 255  
    img = np.array([img,]) 
    prediction = model.predict(img)
    thresholded = (prediction>0.4)*1
    #print(prediction[0][1])
    print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0]) #np.where(thresholded == 1)[1][0]
    #out2 = out2 = (model.predict(img)>0.4)*1

    #print(str(out2))


F:/dataset\H$N0.png

Predicted Digit:
 4
F:/dataset\H$N1.png

Predicted Digit:
 4
F:/dataset\H$N10.png

Predicted Digit:
 4
F:/dataset\H$N100.png

Predicted Digit:
 4
F:/dataset\H$N1000.png

Predicted Digit:
 4
F:/dataset\H$N1001.png

Predicted Digit:
 4
F:/dataset\H$N1002.png

Predicted Digit:
 4
F:/dataset\H$N1003.png

Predicted Digit:
 4
F:/dataset\H$N1004.png

Predicted Digit:
 4
F:/dataset\H$N1005.png

Predicted Digit:
 4
F:/dataset\H$N1006.png

Predicted Digit:
 4
F:/dataset\H$N1007.png

Predicted Digit:
 4
F:/dataset\H$N1008.png

Predicted Digit:
 4
F:/dataset\H$N1009.png

Predicted Digit:
 4
F:/dataset\H$N101.png

Predicted Digit:
 4
F:/dataset\H$N1010.png

Predicted Digit:
 4
F:/dataset\H$N1011.png

Predicted Digit:
 4
F:/dataset\H$N1012.png

Predicted Digit:
 4
F:/dataset\H$N1013.png

Predicted Digit:
 4
F:/dataset\H$N1014.png

Predicted Digit:
 4
F:/dataset\H$N1015.png

Predicted Digit:
 4
F:/dataset\H$N1016.png

Predicted Digit:
 4
F:/dataset\H$N1017.png

Predicted Digit:
 


Predicted Digit:
 4
F:/dataset\H$N1167.png

Predicted Digit:
 4
F:/dataset\H$N1168.png

Predicted Digit:
 4
F:/dataset\H$N1169.png

Predicted Digit:
 4
F:/dataset\H$N117.png

Predicted Digit:
 4
F:/dataset\H$N1170.png

Predicted Digit:
 4
F:/dataset\H$N1171.png

Predicted Digit:
 4
F:/dataset\H$N1172.png

Predicted Digit:
 4
F:/dataset\H$N1173.png

Predicted Digit:
 4
F:/dataset\H$N1174.png

Predicted Digit:
 4
F:/dataset\H$N1175.png

Predicted Digit:
 4
F:/dataset\H$N1176.png

Predicted Digit:
 4
F:/dataset\H$N1177.png

Predicted Digit:
 4
F:/dataset\H$N1178.png

Predicted Digit:
 4
F:/dataset\H$N1179.png

Predicted Digit:
 4
F:/dataset\H$N118.png

Predicted Digit:
 4
F:/dataset\H$N1180.png

Predicted Digit:
 4
F:/dataset\H$N1181.png

Predicted Digit:
 4
F:/dataset\H$N1182.png

Predicted Digit:
 4
F:/dataset\H$N1183.png

Predicted Digit:
 4
F:/dataset\H$N1184.png

Predicted Digit:
 4
F:/dataset\H$N1185.png

Predicted Digit:
 4
F:/dataset\H$N1186.png

Predicted Digit:
 4
F:/dataset\H$


Predicted Digit:
 4
F:/dataset\H$N1339.png

Predicted Digit:
 4
F:/dataset\H$N134.png

Predicted Digit:
 4
F:/dataset\H$N1340.png

Predicted Digit:
 4
F:/dataset\H$N1341.png

Predicted Digit:
 4
F:/dataset\H$N1342.png

Predicted Digit:
 4
F:/dataset\H$N1343.png

Predicted Digit:
 4
F:/dataset\H$N1344.png

Predicted Digit:
 4
F:/dataset\H$N1345.png

Predicted Digit:
 4
F:/dataset\H$N1346.png

Predicted Digit:
 4
F:/dataset\H$N1347.png

Predicted Digit:
 4
F:/dataset\H$N1348.png

Predicted Digit:
 4
F:/dataset\H$N1349.png

Predicted Digit:
 4
F:/dataset\H$N135.png

Predicted Digit:
 4
F:/dataset\H$N1350.png

Predicted Digit:
 4
F:/dataset\H$N1351.png

Predicted Digit:
 4
F:/dataset\H$N1352.png

Predicted Digit:
 4
F:/dataset\H$N1353.png

Predicted Digit:
 4
F:/dataset\H$N1354.png

Predicted Digit:
 4
F:/dataset\H$N1355.png

Predicted Digit:
 4
F:/dataset\H$N1356.png

Predicted Digit:
 4
F:/dataset\H$N1357.png

Predicted Digit:
 4
F:/dataset\H$N1358.png

Predicted Digit:
 4
F:/dataset\H$


Predicted Digit:
 4
F:/dataset\H$N1509.png

Predicted Digit:
 4
F:/dataset\H$N151.png

Predicted Digit:
 4
F:/dataset\H$N1510.png

Predicted Digit:
 4
F:/dataset\H$N1511.png

Predicted Digit:
 4
F:/dataset\H$N1512.png

Predicted Digit:
 4
F:/dataset\H$N1513.png

Predicted Digit:
 4
F:/dataset\H$N1514.png

Predicted Digit:
 4
F:/dataset\H$N1515.png

Predicted Digit:
 4
F:/dataset\H$N1516.png

Predicted Digit:
 4
F:/dataset\H$N1517.png

Predicted Digit:
 4
F:/dataset\H$N1518.png

Predicted Digit:
 4
F:/dataset\H$N1519.png

Predicted Digit:
 4
F:/dataset\H$N152.png

Predicted Digit:
 4
F:/dataset\H$N1520.png

Predicted Digit:
 4
F:/dataset\H$N1521.png

Predicted Digit:
 4
F:/dataset\H$N1522.png

Predicted Digit:
 4
F:/dataset\H$N1523.png

Predicted Digit:
 4
F:/dataset\H$N1524.png

Predicted Digit:
 4
F:/dataset\H$N1525.png

Predicted Digit:
 4
F:/dataset\H$N1526.png

Predicted Digit:
 4
F:/dataset\H$N1527.png

Predicted Digit:
 4
F:/dataset\H$N1528.png

Predicted Digit:
 4
F:/dataset\H$


Predicted Digit:
 4
F:/dataset\H$N1680.png

Predicted Digit:
 4
F:/dataset\H$N1681.png

Predicted Digit:
 4
F:/dataset\H$N1682.png

Predicted Digit:
 4
F:/dataset\H$N1683.png

Predicted Digit:
 4
F:/dataset\H$N1684.png

Predicted Digit:
 4
F:/dataset\H$N1685.png

Predicted Digit:
 4
F:/dataset\H$N1686.png

Predicted Digit:
 4
F:/dataset\H$N1687.png

Predicted Digit:
 4
F:/dataset\H$N1688.png

Predicted Digit:
 4
F:/dataset\H$N1689.png

Predicted Digit:
 4
F:/dataset\H$N169.png

Predicted Digit:
 4
F:/dataset\H$N1690.png

Predicted Digit:
 4
F:/dataset\H$N1691.png

Predicted Digit:
 4
F:/dataset\H$N1692.png

Predicted Digit:
 4
F:/dataset\H$N1693.png

Predicted Digit:
 4
F:/dataset\H$N1694.png

Predicted Digit:
 4
F:/dataset\H$N1695.png

Predicted Digit:
 4
F:/dataset\H$N1696.png

Predicted Digit:
 4
F:/dataset\H$N1697.png

Predicted Digit:
 4
F:/dataset\H$N1698.png

Predicted Digit:
 4
F:/dataset\H$N1699.png

Predicted Digit:
 4
F:/dataset\H$N17.png

Predicted Digit:
 4
F:/dataset\H$N

...


Predicted Digit:
 4
F:/dataset\H$N520.png

Predicted Digit:
 4
F:/dataset\H$N521.png

Predicted Digit:
 4
F:/dataset\H$N522.png

Predicted Digit:
 4
F:/dataset\H$N523.png

Predicted Digit:
 4
F:/dataset\H$N524.png

Predicted Digit:
 4
F:/dataset\H$N525.png

Predicted Digit:
 4
F:/dataset\H$N526.png

Predicted Digit:
 4
F:/dataset\H$N527.png

Predicted Digit:
 4
F:/dataset\H$N528.png

Predicted Digit:
 4
F:/dataset\H$N529.png

Predicted Digit:
 4
F:/dataset\H$N53.png

Predicted Digit:
 4
F:/dataset\H$N530.png

Predicted Digit:
 4
F:/dataset\H$N531.png

Predicted Digit:
 4
F:/dataset\H$N532.png

Predicted Digit:
 4
F:/dataset\H$N533.png

Predicted Digit:
 4
F:/dataset\H$N534.png

Predicted Digit:
 4
F:/dataset\H$N535.png

Predicted Digit:
 4
F:/dataset\H$N536.png

Predicted Digit:
 4
F:/dataset\H$N537.png

Predicted Digit:
 4
F:/dataset\H$N538.png

Predicted Digit:
 4
F:/dataset\H$N539.png

Predicted Digit:
 4
F:/dataset\H$N54.png

Predicted Digit:
 4
F:/dataset\H$N540.png

Predicted Di


Predicted Digit:
 4
F:/dataset\H$N695.png

Predicted Digit:
 4
F:/dataset\H$N696.png

Predicted Digit:
 4
F:/dataset\H$N697.png

Predicted Digit:
 4
F:/dataset\H$N698.png

Predicted Digit:
 4
F:/dataset\H$N699.png

Predicted Digit:
 4
F:/dataset\H$N7.png

Predicted Digit:
 4
F:/dataset\H$N70.png

Predicted Digit:
 4
F:/dataset\H$N700.png

Predicted Digit:
 4
F:/dataset\H$N701.png

Predicted Digit:
 4
F:/dataset\H$N702.png

Predicted Digit:
 4
F:/dataset\H$N703.png

Predicted Digit:
 4
F:/dataset\H$N704.png

Predicted Digit:
 4
F:/dataset\H$N705.png

Predicted Digit:
 4
F:/dataset\H$N706.png

Predicted Digit:
 4
F:/dataset\H$N707.png

Predicted Digit:
 4
F:/dataset\H$N708.png

Predicted Digit:
 4
F:/dataset\H$N709.png

Predicted Digit:
 4
F:/dataset\H$N71.png

Predicted Digit:
 4
F:/dataset\H$N710.png

Predicted Digit:
 4
F:/dataset\H$N711.png

Predicted Digit:
 4
F:/dataset\H$N712.png

Predicted Digit:
 4
F:/dataset\H$N713.png

Predicted Digit:
 4
F:/dataset\H$N714.png

Predicted Digi


Predicted Digit:
 4
F:/dataset\H$N87.png

Predicted Digit:
 4
F:/dataset\H$N870.png

Predicted Digit:
 4
F:/dataset\H$N871.png

Predicted Digit:
 4
F:/dataset\H$N872.png

Predicted Digit:
 4
F:/dataset\H$N873.png

Predicted Digit:
 4
F:/dataset\H$N874.png

Predicted Digit:
 4
F:/dataset\H$N875.png

Predicted Digit:
 4
F:/dataset\H$N876.png

Predicted Digit:
 4
F:/dataset\H$N877.png

Predicted Digit:
 4
F:/dataset\H$N878.png

Predicted Digit:
 4
F:/dataset\H$N879.png

Predicted Digit:
 4
F:/dataset\H$N88.png

Predicted Digit:
 4
F:/dataset\H$N880.png

Predicted Digit:
 4
F:/dataset\H$N881.png

Predicted Digit:
 4
F:/dataset\H$N882.png

Predicted Digit:
 4
F:/dataset\H$N883.png

Predicted Digit:
 4
F:/dataset\H$N884.png

Predicted Digit:
 4
F:/dataset\H$N885.png

Predicted Digit:
 4
F:/dataset\H$N886.png

Predicted Digit:
 4
F:/dataset\H$N887.png

Predicted Digit:
 4
F:/dataset\H$N888.png

Predicted Digit:
 4
F:/dataset\H$N889.png

Predicted Digit:
 4
F:/dataset\H$N89.png

Predicted Dig


Predicted Digit:
 1
F:/dataset\kidney104.png

Predicted Digit:
 1
F:/dataset\kidney1040.png

Predicted Digit:
 1
F:/dataset\kidney1041.png

Predicted Digit:
 1
F:/dataset\kidney1042.png

Predicted Digit:
 1
F:/dataset\kidney1043.png

Predicted Digit:
 1
F:/dataset\kidney1044.png

Predicted Digit:
 1
F:/dataset\kidney1045.png

Predicted Digit:
 1
F:/dataset\kidney1046.png

Predicted Digit:
 1
F:/dataset\kidney1047.png

Predicted Digit:
 1
F:/dataset\kidney1048.png

Predicted Digit:
 1
F:/dataset\kidney1049.png

Predicted Digit:
 1
F:/dataset\kidney105.png

Predicted Digit:
 1
F:/dataset\kidney1050.png

Predicted Digit:
 1
F:/dataset\kidney1051.png

Predicted Digit:
 1
F:/dataset\kidney1052.png

Predicted Digit:
 1
F:/dataset\kidney1053.png

Predicted Digit:
 1
F:/dataset\kidney1054.png

Predicted Digit:
 1
F:/dataset\kidney1055.png

Predicted Digit:
 1
F:/dataset\kidney1056.png

Predicted Digit:
 1
F:/dataset\kidney1057.png

Predicted Digit:
 1
F:/dataset\kidney1058.png

Predicted Digi

IndexError: index 0 is out of bounds for axis 0 with size 0

In [35]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2
from glob import glob                                                           
import os
import simpleaudio as sa
src = glob('E:/cancer_3000/skin/*.png')           # correct code
i=0
dest = r'F:/dataset'
for j in src:
    print(j)
    img = cv2.imread(j)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (w, h)) 
    img = img.astype('float32')
    img /= 255  
    img = np.array([img,]) 
    prediction = model.predict(img)
    thresholded = (prediction>0.3)*1
    #print(prediction[0][1])
    print('\nPredicted Digit:\n',np.where(thresholded == 0)[1][0])
    if np.where(thresholded == 1)[1][0] == 2: 
        img = cv2.imread(j)
        cv2.imwrite(dest+'/skin'+str(i)+'.png', img)
        i=i+1
    #np.where(thresholded == 1)[1][0]
    #out2 = out2 = (model.predict(img)>0.4)*1
print('count  = '+str(i))
    #print(str(out2))


E:/cancer_3000/skin\_0024306.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024307.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024308.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024309.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024310.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024311.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024312.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024313.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024314.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024315.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024316.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024317.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024318.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024319.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024320.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024321.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024322.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024323.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024324


Predicted Digit:
 0
E:/cancer_3000/skin\_0024460.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024461.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024462.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024463.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024464.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024465.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024466.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024467.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024468.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024469.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024470.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024471.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024472.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024473.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024474.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024475.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024476.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024477.png

Predicted Digit:
 0
E:/canc

E:/cancer_3000/skin\_0024612.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024613.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024614.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024615.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024616.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024617.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024618.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024619.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024620.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024621.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024622.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024623.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024624.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024625.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024626.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024627.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024628.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024629.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024630

E:/cancer_3000/skin\_0024766.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024767.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024768.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024769.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024770.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024771.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024772.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024773.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024774.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024775.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024776.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024777.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024778.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024779.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024780.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024781.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024782.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024783.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024784


Predicted Digit:
 0
E:/cancer_3000/skin\_0024920.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024921.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024922.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024923.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024924.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024925.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024926.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024927.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024928.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024929.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024930.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024931.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024932.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024933.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024934.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024935.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024936.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0024937.png

Predicted Digit:
 0
E:/canc

...


Predicted Digit:
 0
E:/cancer_3000/skin\_0026602.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026603.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026604.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026605.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026606.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026607.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026608.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026609.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026610.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026611.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026612.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026613.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026614.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026615.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026616.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026617.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026618.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026619.png

Predicted Digit:
 0
E:/canc


Predicted Digit:
 0
E:/cancer_3000/skin\_0026756.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026757.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026758.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026759.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026760.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026761.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026762.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026763.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026764.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026765.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026766.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026767.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026768.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026769.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026770.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026771.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026772.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026773.png

Predicted Digit:
 0
E:/canc


Predicted Digit:
 0
E:/cancer_3000/skin\_0026908.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026909.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026910.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026911.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026912.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026913.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026914.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026915.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026916.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026917.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026918.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026919.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026920.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026921.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026922.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026923.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026924.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0026925.png

Predicted Digit:
 0
E:/canc


Predicted Digit:
 0
E:/cancer_3000/skin\_0027061.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027062.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027063.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027064.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027065.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027066.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027067.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027068.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027069.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027070.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027071.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027072.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027073.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027074.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027075.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027076.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027077.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027078.png

Predicted Digit:
 0
E:/canc

E:/cancer_3000/skin\_0027213.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027214.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027215.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027216.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027217.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027218.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027219.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027220.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027221.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027222.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027223.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027224.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027225.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027226.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027227.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027228.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027229.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027230.png

Predicted Digit:
 0
E:/cancer_3000/skin\_0027231

In [16]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./tum1.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (w, h)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 

prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.05000298 0.07375765 0.00355737 0.16148433 0.7111977 ]

Thresholded Score:
 [0 0 0 0 1]

Predicted Digit:
 4
[[0.05000298 0.07375765 0.00355737 0.16148433 0.7111977 ]]


In [10]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./mal2.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (w, h)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 

prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.11033848 0.88966155]

Thresholded Score:
 [0 1]

Predicted Digit:
 1
[[0.11033848 0.88966155]]


In [11]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./malli1.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (w, h)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 

prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.07820385 0.9217962 ]

Thresholded Score:
 [0 1]

Predicted Digit:
 1
[[0.07820385 0.9217962 ]]


In [12]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./benn.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (w, h)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 

prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.77535504 0.22464494]

Thresholded Score:
 [1 0]

Predicted Digit:
 0
[[0.77535504 0.22464494]]


In [13]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./16896_idx5_x51_y151_class0.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (w, h)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 

prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.9873344  0.01266559]

Thresholded Score:
 [1 0]

Predicted Digit:
 0
[[0.9873344  0.01266559]]


In [13]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./16896_idx5_x201_y801_class1.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (w, h)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 

prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.16841018 0.8315898 ]

Thresholded Score:
 [0 1]

Predicted Digit:
 1
[[0.16841018 0.8315898 ]]


In [16]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./8974_idx5_x3351_y1601_class0.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (50, 50)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 
prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.45006013 0.5499399 ]

Thresholded Score:
 [0 1]

Predicted Digit:
 1
[[0.45006013 0.5499399 ]]


In [17]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('./16569_idx5_x101_y551_class0.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (50, 50)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 
prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.43386343 0.56613654]

Thresholded Score:
 [0 1]

Predicted Digit:
 1
[[0.43386343 0.56613654]]


In [19]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import numpy as np
import cv2

img = cv2.imread('./16569_idx5_x151_y801_class1.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (50, 50)) 
img = img.astype('float32')
img /= 255  

 
img = np.array([img,])
print(str(img.shape)) 
prediction = model.predict(img)
print('Prediction Score:\n',prediction[0])
thresholded = (prediction>0.5)*1
print('\nThresholded Score:\n',thresholded[0])
print('\nPredicted Digit:\n',np.where(thresholded == 1)[1][0])
if(np.where(thresholded == 1)):
    print('Cancer is Malignant');
else:
    print('Cancer is Bengnin');
out2 = model.predict(img)

print(str(out2))


(1, 50, 50, 3)
Prediction Score:
 [0.2963351 0.7036649]

Thresholded Score:
 [0 1]

Predicted Digit:
 1
Cancer is Malignant
[[0.2963351 0.7036649]]


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        4736      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 64)        4160      
__________

In [38]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimgv
import matplotlib.pyplot as plt
import cv2
from glob import glob                                                           
import os
import simpleaudio as sa
src = glob('E:/Testing_Cancer/*.png')           # correct code
for j in src:
    img = cv2.imread(j)
    img = cv2.resize(img, (w, h)) 
    cv2.imwrite('E:/Testing_Cancer/'+j[18:-5]+'#.png', img)

AttributeError: module 'cv2.cv2' has no attribute 'COLOR_RBG2BGR'